In [19]:
import sys
import pandas as pd
sys.path.append('/Users/smallhamster/Documents/research/jokesclustering/') # ваш путь до корня проекта
from vector_clustering.data.manager import get_jokes_as_dataframe, read_lines_from_file
from gensim.utils import simple_preprocess
from gensim import models, corpora

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

In [2]:
jokes_frame = get_jokes_as_dataframe()
jokes_frame.head()

,joke_text
0,"Мужчину трудно задеть за живое, но уж если зад..."
1,В нашем кемпинге строго запрещено людям разног...
2,А как хорошо у девушек начинается: любимый: ми...
3,"Одна белка случайно попробовала пиво и поняла,..."
4,ОБЪЯВЛЕНИЕ На время мирового финансового кризи...


### Функции предобработки текста
лемматизаторы, чистка стоп-слов, любой необходимый препроцессинг

In [3]:
def gensim_simple_preprocess(text, min_len=3, max_len=1000000):
    ## приводит к нижнему регистру, удаляет знаки препинания, оставляет слова между (min_length, max_length)
    return ' '.join(simple_preprocess(text, min_len=min_len, max_len=max_len))

In [4]:
def rnnmorph_lemmatizer(text):
    ## альтернатива для pymorphy2: приводит слова к нормальной форме.
    return ' '.join([x.normal_form for x in predictor.predict(text.split())])

In [14]:
stop_words = set([x.split('\n')[0] for x in read_lines_from_file('ru_stop_words.txt')])

def stop_words_remove(text):
    ## удаление из текста стоп-слов на основании списка стоп-слов
    return ' '.join([x for x in text.lower().split() if x not in stop_words])

In [17]:
# определяет порядок вызова функций
cleaners = [rnnmorph_lemmatizer, gensim_simple_preprocess, stop_words_remove]

def apply_to_text(t):
    for func in cleaners:
        t = func(t)
    return t

def apply_to_texts(ts):
    return [apply_to_text(t) for t in ts]

In [ ]:
%% time
preprocessed_collection = apply_to_texts(jokes_frame.joke_text.values)

In [18]:
preprocessed_collection = None

### Построение lda модели

In [20]:
number_topics = 200

In [ ]:
dictionary = corpora.Dictionary(preprocessed_collection) # словарь вида: dictionary[word] : numeric_index
corpus = [dictionary.doc2bow(text) for text in tokenized_data] # каждый документ теперь выглядит так:
# [(12, 3), (25, 2), ... ] - то есть в шутке словоа с индексом 12 встретилось 3 раза, слово с индексом 25 - 2 раза и так далее


In [ ]:
lda = models.LdaModel(corpus=corpus, num_topics=number_topics, id2word=dictionary)